In [1]:
import sys
sys.path.append(r'F:\gitlab\mxdbg\src')
from mxESP32Debugger.debugger import Dbg as MXDBG

import time
import ctypes

import toml
from tqdm import trange
from loguru import logger

# Init MXDBG

In [2]:
dev = MXDBG()
dev.usb_config(crc_enable=False)
dev.power_init()
dev.power_control(communication_type="I2C", power_type="3V3")

ret, data = dev.i2c_config(sda_pin=10, scl_pin=11, freq=400000, sda_pullup=True, scl_pullup=True)
print(ret, data)

2025-04-21 12:32:04.276 | INFO     | mxESP32Debugger.debugger:connect:419 - Using serial port to connect. Port: COM4
2025-04-21 12:32:04.277 | INFO     | mxESP32Debugger.debugger:connect:424 - Using ESP32-S3R8.
2025-04-21 12:32:04.278 | INFO     | mxESP32Debugger.debugger:connect:425 - Embedded software version: v1.0
2025-04-21 12:32:04.279 | INFO     | mxESP32Debugger.debugger:connect:426 - Library version: v241210.0.rc3.
2025-04-21 12:32:04.325 | INFO     | mxESP32Debugger.debugger:get_extboard_version:2174 - I2C config for extension board.
2025-04-21 12:32:04.348 | INFO     | mxESP32Debugger.debugger:get_extboard_version:2191 - Extension board version: v0.2.1


True None


# Read configuration files

In [3]:
with open("adxl345.toml", "r") as f:
    config = toml.load(f)
    
reg_list = config['register_list']
slave_addr = config['address']

# Init Sensor

In [4]:
ret, data = dev.i2c_write_read(port=0, slave_id=slave_addr, write_list=[0x00], read_length=1)
assert data[0] == 0xE5, f"Expected 0xE5, got {data[0]:#04x}"

In [5]:
for reg in reg_list:
    ret, data = dev.i2c_write_read(port=0, slave_id=slave_addr, write_list=reg, read_length=0)
    assert ret == True, f"Failed to write register {reg:#04x}"

# Read gravity data

In [6]:
def get_gravity_data(slave_addr, repeat=40, repeat_delay_us=20) -> tuple:
    ret, data = dev.i2c_write_read(port=0, slave_id=slave_addr, write_list=[0x32], read_length=6, repeat=repeat, repeat_delay_us=repeat_delay_us)
    if ret != True:
        print("Error reading data")
        return None, None, None

    data = bytearray(data)

    if repeat != 0:
        X = []
        Y = []
        Z = []
        for i in range(repeat):
            x = ctypes.c_int16.from_buffer_copy(data[i*6+0:i*6+2]).value * 3.9
            y = ctypes.c_int16.from_buffer_copy(data[i*6+2:i*6+4]).value * 3.9
            z = ctypes.c_int16.from_buffer_copy(data[i*6+4:i*6+6]).value * 3.9

            X.append(x)
            Y.append(y)
            Z.append(z)
        return X, Y, Z
    else:
        x = ctypes.c_int16.from_buffer_copy(data[0:2]).value * 3.9
        y = ctypes.c_int16.from_buffer_copy(data[2:4]).value * 3.9
        z = ctypes.c_int16.from_buffer_copy(data[4:6]).value * 3.9

        return x, y, z

In [17]:
import plotly.graph_objects as go
import plotly.express as px
import numpy as np

X=[]
Y=[]
Z=[]

start_time = time.time()
for i in trange(5):
    x, y, z = get_gravity_data(slave_addr=slave_addr, repeat=128, repeat_delay_us=10)
    X.extend(x)
    Y.extend(y)
    Z.extend(z)
end_time = time.time()
timestamp = np.arange(0, end_time - start_time, (end_time - start_time) / len(X))

# add filter for data
# Convert lists to numpy arrays for processing
X = np.array(X)
Y = np.array(Y)
Z = np.array(Z)

# Apply a moving average filter using np.convolve
# The filter kernel is a uniform kernel of size 5 (1/5 for each element)
X = np.convolve(X, np.ones(5)/10, mode='same')
Y = np.convolve(Y, np.ones(5)/10, mode='same')
Z = np.convolve(Z, np.ones(5)/10, mode='same')

fig = go.Figure()
fig.add_trace(go.Scatter(x=timestamp, y=X, mode='lines', name='X-axis'))
fig.add_trace(go.Scatter(x=timestamp, y=Y, mode='lines', name='Y-axis'))
fig.add_trace(go.Scatter(x=timestamp, y=Z, mode='lines', name='Z-axis'))
fig.update_layout(title='ADXL345 Data', xaxis_title='Sample Time Elapsed(s)', yaxis_title='Gravity(mg/s^2)')
fig.show()

100%|██████████| 5/5 [00:06<00:00,  1.29s/it]
